In [0]:
from pyspark.sql.functions import col, explode, lower, regexp_extract, regexp_replace, split, trim

# Read invoice files

In [0]:
base_path = '/Volumes/workspace/default/raw_invoices_pdf'
files = dbutils.fs.ls(base_path)
display(files)

path,name,size,modificationTime
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf,sample_invoice_usd_1.pdf,2569,1771065183000
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf,sample_invoice_usd_2.pdf,2592,1771065183000
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf,sample_invoice_usd_3.pdf,2569,1771065183000
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_4.pdf,sample_invoice_usd_4.pdf,2612,1771065183000


# Filter PDF files

In [0]:
raw_df = (spark
    .read.format("binaryFile").option("pathGlobFilter", "*.pdf").load(base_path)
    .select("path", "content", "length", "modificationTime")
)
display(raw_df)

path,content,length,modificationTime
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_4.pdf,JVBERi0xLjQKJZOMi54gUmVwb3J0TGFiIEdlbmVyYXRlZCBQREYgZG9jdW1lbnQgaHR0cDovL3d3dy5yZXBvcnRsYWIuY29tCjEgMCBvYmoKPDwKL0YxIDIgMCBSIC9GMiAzIDAgUgo+PgplbmRvYmoKMiAwIG9iago8PAovQmFzZUZvbnQgL0hlbHZldGljYSAvRW5jb2RpbmcgL1dpbkFuc2lFbmNvZGluZyAvTmFtZSAvRjEgL1N1YnR5cGUgL1R5cGUxIC9UeXBlIC9Gb250Cj4+CmVuZG9iagozIDAgb2JqCjw8Ci9CYXNlRm9udCAvSGVsdmV0aWNhLUJvbGQgL0VuY29kaW5nIC9XaW5BbnNpRW5jb2RpbmcgL05hbWUgL0YyIC9TdWJ0eXBlIC9UeXBlMSAvVHlwZSAvRm9udAo+PgplbmRvYmoKNCAwIG9iago8PAovQ29udGVudHMgOCAwIFIgL01lZGlhQm94IFsgMCAwIDU5NS4yNzU2IDg0MS44ODk4IF0gL1BhcmVudCA3IDAgUiAvUmVzb3VyY2VzIDw8Ci9Gb250IDEgMCBSIC9Qcm9jU2V0IFsgL1BERiAvVGV4dCAvSW1hZ2VCIC9JbWFnZUMgL0ltYWdlSSBdCj4+IC9Sb3RhdGUgMCAvVHJhbnMgPDwKCj4+IAogIC9UeXBlIC9QYWdlCj4+CmVuZG9iago1IDAgb2JqCjw8Ci9QYWdlTW9kZSAvVXNlTm9uZSAvUGFnZXMgNyAwIFIgL1R5cGUgL0NhdGFsb2cKPj4KZW5kb2JqCjYgMCBvYmoKPDwKL0F1dGhvciAoXChhbm9ueW1vdXNcKSkgL0NyZWF0aW9uRGF0ZSAoRDoyMDI2MDIxNDEwMDg0MSswMCcwMCcpIC9DcmVhdG9yIChcKHVuc3BlY2lmaWVkXCkpIC9LZXl3b3JkcyAoKSAvTW9kRGF0ZSAoRDoyMDI2MDIxNDEwMDg0MSswMCcwMCcpIC9Qcm9kdWNlciAoUmVwb3J0TGFiIFBERiBMaWJyYXJ5IC0gd3d3LnJlcG9ydGxhYi5jb20pIAogIC9TdWJqZWN0IChcKHVuc3BlY2lmaWVkXCkpIC9UaXRsZSAoXChhbm9ueW1vdXNcKSkgL1RyYXBwZWQgL0ZhbHNlCj4+CmVuZG9iago3IDAgb2JqCjw8Ci9Db3VudCAxIC9LaWRzIFsgNCAwIFIgXSAvVHlwZSAvUGFnZXMKPj4KZW5kb2JqCjggMCBvYmoKPDwKL0ZpbHRlciBbIC9BU0NJSTg1RGVjb2RlIC9GbGF0ZURlY29kZSBdIC9MZW5ndGggMTE3MAo+PgpzdHJlYW0KR2F0JSJnTik9NCUiNkgnVFtTIyZdOF1zIjcjTFA2UDtxPG9QTzBeJWc6bG5dRCk4RlRaSytXIjtaRC5nPVZrRWpWXUNhSyQ5LyQqNV8oKl5NQFY2O0IxbWNVJjpYSm9KOGtqZUo5Zihjbi4nP1FaYk9nZ0wlSUEiXi8pKkcxZzZRKFFBWSYpJHVLLy0lTG5jZ00/RD1dLydcTW1TOCJdVylIJGsoMFklKVtjUyljWjJdajE/IzJyJmhtXTE/WSNDUTVYUTlkPXVTazlWQUhqcl1IVCNhTiladC4uOjMlNURSQ1UyIXMwMWchTVhhVFJPTU80WEBaPy8pVikzZ2FwPW5LQEdKIWpAR2Y0Z0RPQGkkSDtFLUhOXjZAOWZoQjlVIzIjRVBqUFt1LSM2P2xba19vRSkwSytsSjxiPGRER04nMENkZjBqIzNBUFMsQ25dW2JtajliYFooYlwkKiNFVltyMDE2PUYnS0ssSTUjJ2NBNC9wMEpNUXAjJ08/MXRiTENEdEdLSHA3WztiVFdiMkNJOSFaTENuVU1lV0I+RmdYV0FXb1A6WSkzLzAtQ2FtLDNEOGVaSFYlUVMhKThaVFw9cVVLLSoxa1RjYF4iUU0sUVhcZ1RYSTFaS2ZxYikpQS4uIy9SQCwhLyRQPThsYmUjXHJjSSdPTCJZLk1SLm83X1QnYnIsI2g1Tlo2XGFOKW9MTXM5OSJPL2IuOk5MP1gra01mWUdrdVldWmklJT5HPzBNbWlhWSleTj1LaXM/Lk9HMiVxPltRMzA2IjpEIkNkRlhXL1twKXBgcEA4Ml4vSVwqOm09KlslVj9ySzBKWG1QQ3F1RFJARzUmWmtNS1xMYWImKzU1LDkzdDtzTFg+SGJSPHVhcWNhZmNnZVRqL29raFJJNmJfbzAuWldlWUpiJDImJlYqbkxKZihXZ0k+XFtwXmAma1tNTCQ6XVhJJDxGcjI3PnBvZ1daWWY4MDtiX2pqdE82STJHUj1ISF5NXFE8Z0AqU2tERTdDM2JLOSpJOSY0LT5jPGpmQTQ5LF9QMWgqREBqPFZjJTg+P1I8TFRJaiw+VHFTJ0xeUkVTZDIwXEdhLl1IQUM4aT9pVToiJE1OPk8xYSc3QiZiR251aihyTi82ZFJIZSVBQClOamw0LlVvMixZRkY7RVR0Uy4xVSxCKmk2N1lMS2MzRiFKT20uMiw6XjtiKTFCZidKcHI0J1E2TEQsLGlGRyxPRihWLFRiMzlnUl0mNEtAZl9vYTIsQXAuQjZMK21MbmJdJCt1V3I7NWplYXAhZmhgYDFycGNBbHVTOTFnJmwzX0VWMjZLKG06NlZcNHEkPEVmciJvKy0zaGtUQE5ma19mXmpEVmArNSFXc1duX3BRMEYqLCYoaCg0biY3Z0Mxc25lInMqXTEkbkxhbjJjViFPKjtcKEtvNDklRGk5IWk+TVFFSDYrQVZqJForNks0US09JW1nUSlaNEEnRl1JZz5fUTI3XEY7SCpPUy9ScG4lKFwmXi5xIlBnbDIqM1UzI11PSHRuYUdbIlVsc0NNMEVGJn4+ZW5kc3RyZWFtCmVuZG9iagp4cmVmCjAgOQowMDAwMDAwMDAwIDY1NTM1IGYgCjAwMDAwMDAwNzMgMDAwMDAgbiAKMDAwMDAwMDExNCAwMDAwMCBuIAowMDAwMDAwMjIxIDAwMDAwIG4gCjAwMDAwMDAzMzMgMDAwMDAgbiAKMDAwMDAwMDUzNiAwMDAwMCBuIAowMDAwMDAwNjA0IDAwMDAwIG4gCjAwMDAwMDA4ODcgMDAwMDAgbiAKMDAwMDAwMDk0NiAwMDAwMCBuIAp0cmFpbGVyCjw8Ci9JRCAKWzxhNzRkNWJjM2I2NjFkZmI4MjdiOGQyNWMwMTg0ZTg5NT48YTc0ZDViYzNiNjYxZGZiODI3YjhkMjVjMDE4NGU4OTU+XQolIFJlcG9ydExhYiBnZW5lcmF0ZWQgUERGIGRvY3VtZW50IC0tIGRpZ2VzdCAoaHR0cDovL3d3dy5yZXBvcnRsYWIuY29tKQoKL0luZm8gNiAwIFIKL1Jvb3QgNSAwIFIKL1NpemUgOQo+PgpzdGFydHhyZWYKMjIwNwolJUVPRgo=,2612,2026-02-14T10:33:03.000Z
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf,JVBERi0xLjQKJZOMi54gUmVwb3J0TGFiIEdlbmVyYXRlZCBQREYgZG9jdW1lbnQgaHR0cDovL3d3dy5yZXBvcnRsYWIuY29tCjEgMCBvYmoKPDwKL0YxIDIgMCBSIC9GMiAzIDAgUgo+PgplbmRvYmoKMiAwIG9iago8PAovQmFzZUZvbnQgL0hlbHZldGljYSAvRW5jb2RpbmcgL1dpbkFuc2lFbmNvZGluZyAvTmFtZSAvRjEgL1N1YnR5cGUgL1R5cGUxIC9UeXBlIC9Gb250Cj4+CmVuZG9iagozIDAg

In [0]:
parsed_pdf_df = raw_df.select(
    "path",
    expr("ai_parse_document(content) as parsed_document")
)
display(parsed_pdf_df)

path parsed_document dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_4.pdf {"document":{"elements":[{"bbox":[{"coord":[434,72,562,105],"page_id":0}],"content":"INVOICE","description":null,"id":0,"type":"section_header"},{"bbox":[{"coord":[81,160,339,288],"page_id":0}],"content":"From:\nBrightWave Consulting LLC\n789 Market Road\nSan Diego, CA 92101\nPhone: +1 619 555 0182\nEmail: accounts@brightwave.test","description":null,"id":1,"type":"text"},{"bbox":[{"coord":[499,160,683,326],"page_id":0}],"content":"Invoice #: INV-1004\nDate: 2026-02-14\nDue Date: 2026-02-14\nBill To:\nOrion Retail Co.\n22 Harbor Drive\nLong Beach, CA 90802\nContact: Maria Santos","description":null,"id":2,"type":"text"},{"bbox":[{"coord":[93,381,899,520],"page_id":0}],"content":" Item Description Qty Unit Price Amount 001 Retail Analytics Setup 1 $6,200.00 $6,200.00 002 Dashboard Development 2 $1,800.00 $3,600.00 003 Monthly Support 1 $900.00 $900.00 ","description":null,"id":3,"type":"table"},{"bbox":[{"coord":[627,545,880,653],"page_id":0}],"content":" Subtotal: $10,700.00 Tax (8%): $909.50 Total: $11,609.50 ","description":null,"id":4,"type":"table"},{"bbox":[{"coord":[66,697,606,723],"page_id":0}],"content":"Thank you for your business! Please make payment within the due date.","description":null,"id":5,"type":"text"}],"pages":[{"id":0,"image_uri":null}]},"error_status":null,"metadata":{"file_metadata":null,"id":"bb043637-3839-497a-a842-7e62f8a9da0a","version":"2.0"}} dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf {"document":{"elements":[{"bbox":[{"coord":[434,72,562,105],"page_id":0}],"content":"INVOICE","description":null,"id":0,"type":"section_header"},{"bbox":[{"coord":[81,160,307,287],"page_id":0}],"content":"From:\nNorthPeak Systems Inc.\n55 Innovation Ave\nAustin, TX 78701\nPhone: +1 512 555 7744\nEmail: billing@northpeak.test","description":null,"id":1,"type":"text"},{"bbox":[{"coord":[503,160,676,326],"page_id":0}],"content":"Invoice #: INV-1005\nDate: 2026-02-14\nDue Date: 2026-02-14\nBill To:\nSummit Logistics Ltd.\n310 Industrial Park\nPhoenix, AZ 85004\nContact: Daniel Cruz","description":null,"id":2,"type":"text"},{"bbox":[{"coord":[93,381,899,520],"page_id":0}],"content":" Item Description Qty Unit Price Amount 001 API Integration 1 $4,500.00 $4,500.00 002 Workflow Automation 3 $1,600.00 $4,800.00 003 User Training 1 $1,200.00 $1,200.00 ","description":null,"id":3,"type":"table"},{"bbox":[{"coord":[627,545,880,651],"page_id":0}],"content":" Subtotal: $10,500.00 Tax (7%): $787.50 Total: $11,287.50 ","description":null,"id":4,"type":"table"},{"bbox":[{"coord":[66,697,606,723],"page_id":0}],"content":"Thank you for your business! Please make payment within the due date.","description":null,"id":5,"type":"text"}],"pages":[{"id":0,"image_uri":null}]},"error_status":null,"metadata":{"file_metadata":null,"id":"5892a8e9-107e-406f-a4d3-dfb7c3e8c9c9","version":"2.0"}} dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf {"document":{"elements":[{"bbox":[{"coord":[434,72,562,105],"page_id":0}],"content":"INVOICE","description":null,"id":0,"type":"section_header"},{"bbox":[{"coord":[81,160,339,287],"page_id":0}],"content":"From:\nAcme Solutions Inc.\n123 Business Street\nMetro City, PH 1000\nPhone: +63 900 000 0000\nEmail: billing@acmesolutions.test","description":null,"id":1,"type":"text"},{"bbox":[{"coord":[503,160,686,305],"page_id":0}],"content":"Invoice #: INV-1001\nDate: 2026-02-14\nDue Date: 2026-02-14\nBill To:\nJohn Doe\n456 Client Avenue\nSample Town, PH 2000","description":null,"id":2,"type":"text"},{"bbox":[{"coord":[93,360,899,500],"page_id":0}],"content":" Item Description Qty Unit Price Amount 001 Data Pipeline Setup 1 $10,000.00 $10,000.00 002 Automation Workflow 2 $3,500.00 $7,000.00 003 Support & Maintenance 1 $2,000.00 $2,000.00 ","description":null,"id":3,"type":"table"},{"bbox":[{"coord":[610,525,880,631],"page_id":0}],"content":" Subtotal: $19,000.00 Tax (1

In [0]:
elements_df = parsed_pdf_df.select(
    "path",
    expr("cast(variant_get(parsed_document, '$.document.elements') as string) as elements_json")
)
display(elements_df)

path elements_json dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_4.pdf [{"bbox":[{"coord":[434,72,562,106],"page_id":0}],"content":"INVOICE","description":null,"id":0,"type":"section_header"},{"bbox":[{"coord":[83,161,339,288],"page_id":0}],"content":"From:\nBrightWave Consulting LLC\n789 Market Road\nSan Diego, CA 92101\nPhone: +1 619 555 0182\nEmail: accounts@brightwave.test","description":null,"id":1,"type":"text"},{"bbox":[{"coord":[503,161,682,326],"page_id":0}],"content":"Invoice #: INV-1004\nDate: 2026-02-14\nDue Date: 2026-02-14\nBill To:\nOrion Retail Co.\n22 Harbor Drive\nLong Beach, CA 90802\nContact: Maria Santos","description":null,"id":2,"type":"text"},{"bbox":[{"coord":[91,379,901,521],"page_id":0}],"content":" Item Description Qty Unit Price Amount 001 Retail Analytics Setup 1 $6,200.00 $6,200.00 002 Dashboard Development 2 $1,800.00 $3,600.00 003 Monthly Support 1 $900.00 $900.00 ","description":null,"id":3,"type":"table"},{"bbox":[{"coord":[627,546,880,653],"page_id":0}],"content":" Subtotal: $10,700.00 Tax (8%): $909.50 Total: $11,609.50 ","description":null,"id":4,"type":"table"},{"bbox":[{"coord":[66,697,606,723],"page_id":0}],"content":"Thank you for your business! Please make payment within the due date.","description":null,"id":5,"type":"text"}] dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf [{"bbox":[{"coord":[434,72,562,105],"page_id":0}],"content":"INVOICE","description":null,"id":0,"type":"section_header"},{"bbox":[{"coord":[81,160,307,287],"page_id":0}],"content":"From:\nNorthPeak Systems Inc.\n55 Innovation Ave\nAustin, TX 78701\nPhone: +1 512 555 7744\nEmail: billing@northpeak.test","description":null,"id":1,"type":"text"},{"bbox":[{"coord":[503,160,676,326],"page_id":0}],"content":"Invoice #: INV-1005\nDate: 2026-02-14\nDue Date: 2026-02-14\nBill To:\nSummit Logistics Ltd.\n310 Industrial Park\nPhoenix, AZ 85004\nContact: Daniel Cruz","description":null,"id":2,"type":"text"},{"bbox":[{"coord":[93,381,899,520],"page_id":0}],"content":" Item Description Qty Unit Price Amount 001 API Integration 1 $4,500.00 $4,500.00 002 Workflow Automation 3 $1,600.00 $4,800.00 003 User Training 1 $1,200.00 $1,200.00 ","description":null,"id":3,"type":"table"},{"bbox":[{"coord":[627,545,880,651],"page_id":0}],"content":" Subtotal: $10,500.00 Tax (7%): $787.50 Total: $11,287.50 ","description":null,"id":4,"type":"table"},{"bbox":[{"coord":[66,697,606,723],"page_id":0}],"content":"Thank you for your business! Please make payment within the due date.","description":null,"id":5,"type":"text"}] dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf [{"bbox":[{"coord":[434,72,562,105],"page_id":0}],"content":"INVOICE","description":null,"id":0,"type":"section_header"},{"bbox":[{"coord":[81,160,339,287],"page_id":0}],"content":"From:\nAcme Solutions Inc.\n123 Business Street\nMetro City, PH 1000\nPhone: +63 900 000 0000\nEmail: billing@acmesolutions.test","description":null,"id":1,"type":"text"},{"bbox":[{"coord":[503,160,686,305],"page_id":0}],"content":"Invoice #: INV-1001\nDate: 2026-02-14\nDue Date: 2026-02-14\nBill To:\nJohn Doe\n456 Client Avenue\nSample Town, PH 2000","description":null,"id":2,"type":"text"},{"bbox":[{"coord":[93,360,899,500],"page_id":0}],"content":" Item Description Qty Unit Price Amount 001 Data Pipeline Setup 1 $10,000.00 $10,000.00 002 Automation Workflow 2 $3,500.00 $7,000.00 003 Support & Maintenance 1 $2,000.00 $2,000.00 ","description":null,"id":3,"type":"table"},{"bbox":[{"coord":[610,526,880,631],"page_id":0}],"content":" Subtotal: $19,000.00 Tax (12%): $2,280.00 Total: $21,280.00 ","description":null,"id":4,"type":"table"},{"bbox":[{"coord":[66,678,606,702],"page_id":0}],"content":"Thank you for your business! Please make payment within the due date.","description":null,"id":5,"type":"text"}] dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf [{"bbox":[{"coord":[434,72,562,105],"page_id":0}],"con

In [0]:
elements_schema = """
array<struct<
    id: int,
    type: string,
    content: string,
    bbox: array<struct<coord: array<int>, page_id: int>>,
    description: string
>>
"""

elements_df = (
    elements_df
    .select(
        "path",
        explode(from_json(col("elements_json"), elements_schema)).alias("invoice_info")
    )
    .select(
        "path",
        # col("invoice_info"),
        col("invoice_info.id").alias("element_id"),
        col("invoice_info.content").alias("element_content"),
        col("invoice_info.type").alias("element_type"),
        col("invoice_info.bbox").alias("element_bbox"),
        col("invoice_info.description").alias("element_description")
    )
    .orderBy("path", "element_id")
)
display(elements_df)


path element_id element_content element_type element_bbox element_description dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf 0 INVOICE section_header List(List(List(434, 72, 562, 105), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf 1 From:
Acme Solutions Inc.
123 Business Street
Metro City, PH 1000
Phone: +63 900 000 0000
Email: billing@acmesolutions.test text List(List(List(81, 160, 339, 287), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf 2 Invoice #: INV-1001
Date: 2026-02-14
Due Date: 2026-02-14
Bill To:
John Doe
456 Client Avenue
Sample Town, PH 2000 text List(List(List(503, 160, 686, 305), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf 3 Item Description Qty Unit Price Amount 001 Data Pipeline Setup 1 $10,000.00 $10,000.00 002 Automation Workflow 2 $3,500.00 $7,000.00 003 Support & Maintenance 1 $2,000.00 $2,000.00 table List(List(List(93, 360, 899, 500), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf 4 Subtotal: $19,000.00 Tax (12%): $2,280.00 Total: $21,280.00 table List(List(List(610, 526, 880, 631), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf 5 Thank you for your business! Please make payment within the due date. text List(List(List(66, 678, 606, 702), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf 0 INVOICE section_header List(List(List(434, 72, 562, 105), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf 1 From:
NorthPeak Systems Inc.
55 Innovation Ave
Austin, TX 78701
Phone: +1 512 555 7744
Email: billing@northpeak.test text List(List(List(81, 160, 307, 287), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf 2 Invoice #: INV-1005
Date: 2026-02-14
Due Date: 2026-02-14
Bill To:
Summit Logistics Ltd.
310 Industrial Park
Phoenix, AZ 85004
Contact: Daniel Cruz text List(List(List(503, 160, 676, 326), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf 3 Item Description Qty Unit Price Amount 001 API Integration 1 $4,500.00 $4,500.00 002 Workflow Automation 3 $1,600.00 $4,800.00 003 User Training 1 $1,200.00 $1,200.00 table List(List(List(93, 381, 899, 520), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf 4 Subtotal: $10,500.00 Tax (7%): $787.50 Total: $11,287.50 table List(List(List(627, 545, 880, 651), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf 5 Thank you for your business! Please make payment within the due date. text List(List(List(66, 697, 606, 723), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf 0 INVOICE section_header List(List(List(434, 72, 562, 106), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf 1 From:
Acme Solutions Inc.
123 Business Street
Metro City, PH 1000
Phone: +63 900 000 0000
Email: billing@acmesolutions.test text List(List(List(81, 160, 339, 288), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf 2 Invoice #: INV-1003
Date: 2026-02-14
Due Date: 2026-02-14
Bill To:
Robert Lee
456 Client Avenue
Sample Town, PH 2000 text List(List(List(503, 160, 686, 305), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf 3 Item Description Qty Unit Price Amount 001 AI Automation Design 1 $8,000.00 $8,000.00 002 Workflow Implementation 3 $2,200.00 $6,600.00 003 Training & Handover 1 $1,800.00 $1,800.00 table List(List(List(93, 359, 899, 501), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf 4 Subtotal: $16,400.00 Tax (12%): $1,968.00 Total: $18,368.00 table List(List(List(611, 528, 880, 633), 0)) null dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf 5 Thank you for your business! Please make payment within the due

# Sender Information

In [0]:
keywords = ["from", "phone", "email"]

sender_df = (
    elements_df
    .withColumn("content_l", lower(col("element_content")))
    .where(" OR ".join([f"content_l LIKE '%{k}%'" for k in keywords]))
    .select("path", "element_type", "element_content", "element_bbox", "element_id")
    .orderBy("path", "element_id")
    .where(col("element_content").isNotNull())
    # Extract everything from "From:" up to "Phone:" (multi-line safe)
    .withColumn(
        "from_raw",
        regexp_extract(
            col("element_content"),
            r"(?s)From:\s*(.*?)\s*Phone:",
            1
        )
    )
    # Extract phone (supports spaces, +, -, (), etc.)
    .withColumn(
        "phone",
        regexp_extract(
            col("element_content"),
            r"Phone:\s*([+\d][\d\s\-()]+)",
            1
        )
    )
    # Extract email
    .withColumn(
        "email",
        regexp_extract(
            col("element_content"),
            r"Email:\s*([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})",
            1
        )
    )
    # Clean up whitespace and line breaks at the ends
    .withColumn("from", trim(col("from_raw")))
    .drop("from_raw")
    # Select final columns
    .select("path", "from", "phone", "email")
)

display(sender_df)


path,from,phone,email
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf,"Acme Solutions Inc. 123 Business Street Metro City, PH 1000",+63 900 000 0000,billing@acmesolutions.test
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf,"NorthPeak Systems Inc. 55 Innovation Ave Austin, TX 78701",+1 512 555 7744,billing@northpeak.test
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf,"Acme Solutions Inc. 123 Business Street Metro City, PH 1000",+63 900 000 0000,billing@acmesolutions.test
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_4.pdf,"BrightWave Consulting LLC 789 Market Road San Diego, CA 92101",+1 619 555 0182,accounts@brightwave.test


# Invoice Header

In [0]:
keywords = ["invoice", "date", "due date", "bill to"]

invoice_header_df = (
    elements_df
    .withColumn("content_l", lower(col("element_content")))
    .where(" OR ".join([f"content_l LIKE '%{k}%'" for k in keywords]))
    .select("path", "element_type", "element_content", "element_bbox", "element_id")
    .orderBy("path", "element_id")
    .where(col("element_content").isNotNull())
    # Invoice number
    .withColumn(
        "invoice_no",
        regexp_extract(
            col("element_content"),
            r"Invoice\s*#:\s*([A-Za-z0-9\-]+)",
            1
        )
    )
    # Date
    .withColumn(
        "date",
        regexp_extract(
            col("element_content"),
            r"Date:\s*([0-9]{4}-[0-9]{2}-[0-9]{2})",
            1
        )
    )
    # Due Date
    .withColumn(
        "due_date",
        regexp_extract(
            col("element_content"),
            r"Due Date:\s*([0-9]{4}-[0-9]{2}-[0-9]{2})",
            1
        )
    )
    # Bill To: everything after "Bill To:" (multi-line safe)
    .withColumn(
        "bill_to_raw",
        regexp_extract(
            col("element_content"),
            r"(?s)Bill To:\s*(.*)$",
            1
        )
    )
    # Clean up whitespace
    .withColumn("bill_to", trim(col("bill_to_raw")))
    .drop("bill_to_raw")
    # Select final columns
    .select("path", "element_content", "invoice_no", "date", "due_date", "bill_to")
    .where("invoice_no <> ''")
)

display(invoice_header_df)


path,element_content,invoice_no,date,due_date,bill_to
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf,"Invoice #: INV-1001 Date: 2026-02-14 Due Date: 2026-02-14 Bill To: John Doe 456 Client Avenue Sample Town, PH 2000",INV-1001,2026-02-14,2026-02-14,"John Doe 456 Client Avenue Sample Town, PH 2000"
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf,"Invoice #: INV-1005 Date: 2026-02-14 Due Date: 2026-02-14 Bill To: Summit Logistics Ltd. 310 Industrial Park Phoenix, AZ 85004 Contact: Daniel Cruz",INV-1005,2026-02-14,2026-02-14,"Summit Logistics Ltd. 310 Industrial Park Phoenix, AZ 85004 Contact: Daniel Cruz"
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf,"Invoice #: INV-1003 Date: 2026-02-14 Due Date: 2026-02-14 Bill To: Robert Lee 456 Client Avenue Sample Town, PH 2000",INV-1003,2026-02-14,2026-02-14,"Robert Lee 456 Client Avenue Sample Town, PH 2000"
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_4.pdf,"Invoice #: INV-1004 Date: 2026-02-14 Due Date: 2026-02-14 Bill To: Orion Retail Co. 22 Harbor Drive Long Beach, CA 90802 Contact: Maria Santos",INV-1004,2026-02-14,2026-02-14,"Orion Retail Co. 22 Harbor Drive Long Beach, CA 90802 Contact: Maria Santos"


# Line Item

In [0]:
keywords = ["item", "description", "qty", "unit price", "amount"]

line_items_df = (
    elements_df
    .withColumn("content_l", lower(col("element_content")))
    .where(" OR ".join([f"content_l LIKE '%{k}%'" for k in keywords]))
    .select("path", "element_type", "element_content", "element_bbox", "element_id")
    .orderBy("path", "element_id")
    .where(col("element_content").isNotNull())
    # Remove newlines just in case
    .withColumn("tbl", regexp_replace(col("element_content"), r"\n", ""))
    # Split into rows by </tr>
    .withColumn("rows", split(col("tbl"), r"</tr>"))
    .withColumn("row", explode(col("rows")))
    # Keep only rows that actually contain <td> (skip header)
    .where(col("row").like("%<td>%"))
    # Extract each cell by position
    .withColumn("item", 
        regexp_extract(col("row"), r"<td>(.*?)</td>", 1)
    )
    .withColumn("description",
        regexp_extract(col("row"), r"<td>.*?</td><td>(.*?)</td>", 1)
    )
    .withColumn("qty", 
        regexp_extract(col("row"), r"<td>.*?</td><td>.*?</td><td>(.*?)</td>", 1)
    )
    .withColumn("unit_price_raw",
        regexp_extract(col("row"), r"<td>.*?</td><td>.*?</td><td>.*?</td><td>\$(.*?)</td>", 1)
    )
    .withColumn("amount_raw", 
        regexp_extract(col("row"), r"<td>.*?</td><td>.*?</td><td>.*?</td><td>.*?</td><td>\$(.*?)</td>", 1)
    )
    # Clean numbers (remove commas) and cast
    .withColumn("unit_price",
        regexp_replace(col("unit_price_raw"), ",", "").cast("decimal(18,2)")
    )
    .withColumn("amount", 
        regexp_replace(col("amount_raw"), ",", "").cast("decimal(18,2)")
    )
    .withColumn("qty", col("qty").cast("int"))
    # Select final columns
    .select("path", "item", "description", "qty", "unit_price", "amount")
)

display(line_items_df)


path,item,description,qty,unit_price,amount
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf,001,Data Pipeline Setup,1,10000.00,10000.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf,002,Automation Workflow,2,3500.00,7000.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf,003,Support & Maintenance,1,2000.00,2000.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf,001,API Integration,1,4500.00,4500.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf,002,Workflow Automation,3,1600.00,4800.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf,003,User Training,1,1200.00,1200.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf,001,AI Automation Design,1,8000.00,8000.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf,002,Workflow Implementation,3,2200.00,6600.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf,003,Training & Handover,1,1800.00,1800.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_4.pdf,001,Retail Analytics Setup,1,6200.00,6200.00


# Invoice Amount

In [0]:
keywords = ["subtotal", "tax", "total"]

invoice_amount_df = (
    elements_df
    .withColumn("content_l", lower(col("element_content")))
    .where(" OR ".join([f"content_l LIKE '%{k}%'" for k in keywords]))
    .select("path", "element_type", "element_content", "element_bbox", "element_id")
    .orderBy("path", "element_id")
    # Extract raw strings with currency symbols/commas
    .withColumn(
        "subtotal_raw",
        regexp_extract(
            col("element_content"),
            r"Subtotal:</td><td>\$([0-9,]+\.\d{2})",
            1
        )
    )
    .withColumn(
        "tax_raw",
        regexp_extract(
            col("element_content"),
            r"Tax.*?:</td><td>\$([0-9,]+\.\d{2})",
            1
        )
    )
    .withColumn(
        "total_raw",
        regexp_extract(
            col("element_content"),
            r"Total:</td><td>\$([0-9,]+\.\d{2})",
            1
        )
    )
    # Clean commas and cast to decimal (or keep as string if you prefer)
    .withColumn("subtotal", regexp_replace(col("subtotal_raw"), ",", "").cast("decimal(18,2)"))
    .withColumn("tax", regexp_replace(col("tax_raw"), ",", "").cast("decimal(18,2)"))
    .withColumn("total", regexp_replace(col("total_raw"), ",", "").cast("decimal(18,2)"))
    .drop("subtotal_raw", "tax_raw", "total_raw")
)

display(invoice_amount_df)


path element_type element_content element_bbox element_id subtotal tax total dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf table Subtotal: $19,000.00 Tax (12%): $2,280.00 Total: $21,280.00 List(List(List(610, 525, 880, 633), 0)) 4 19000.00 2280.00 21280.00 dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf table Subtotal: $10,500.00 Tax (7%): $787.50 Total: $11,287.50 List(List(List(627, 546, 880, 650), 0)) 4 10500.00 787.50 11287.50 dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf table Subtotal: $16,400.00 Tax (12%): $1,968.00 Total: $18,368.00 List(List(List(611, 528, 880, 633), 0)) 4 16400.00 1968.00 18368.00 dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_4.pdf table Subtotal: $10,700.00 Tax (8%): $909.50 Total: $11,609.50 List(List(List(627, 546, 880, 653), 0)) 4 10700.00 909.50 11609.50

In [0]:
final_df = (
    invoice_header_df.alias("h")
    .join(sender_df.alias("s"), on="path", how="left")
    .join(invoice_amount_df.alias("a"), on="path", how="left")
    .select("path", "from", "phone", "email", "invoice_no", "date", "due_date", "bill_to", "subtotal", "tax", "total")
    .orderBy("path")
)

display(final_df)

path,from,phone,email,invoice_no,date,due_date,bill_to,subtotal,tax,total
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf,"Acme Solutions Inc. 123 Business Street Metro City, PH 1000",+63 900 000 0000,billing@acmesolutions.test,INV-1001,2026-02-14,2026-02-14,"John Doe 456 Client Avenue Sample Town, PH 2000",19000.00,2280.00,21280.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf,"NorthPeak Systems Inc. 55 Innovation Ave Austin, TX 78701",+1 512 555 7744,billing@northpeak.test,INV-1005,2026-02-14,2026-02-14,"Summit Logistics Ltd. 310 Industrial Park Phoenix, AZ 85004 Contact: Daniel Cruz",10500.00,787.50,11287.50
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf,"Acme Solutions Inc. 123 Business Street Metro City, PH 1000",+63 900 000 0000,billing@acmesolutions.test,INV-1003,2026-02-14,2026-02-14,"Robert Lee 456 Client Avenue Sample Town, PH 2000",16400.00,1968.00,18368.00
dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_4.pdf,"BrightWave Consulting LLC 789 Market Road San Diego, CA 92101",+1 619 555 0182,accounts@brightwave.test,INV-1004,2026-02-14,2026-02-14,"Orion Retail Co. 22 Harbor Drive Long Beach, CA 90802 Contact: Maria Santos",10700.00,909.50,11609.50


# Validate final dataframe

In [0]:
from pyspark.sql.functions import col, when, abs, round

validated_df = (
    final_df
    .withColumn("computed_total", col("subtotal") + col("tax"))
    .withColumn("diff_amount", col("total") - col("computed_total"))
    .withColumn(
        "diff_pct",
        when(col("total").isNotNull() & (col("total") != 0),
             round(col("diff_amount") / col("total") * 100, 2)
        ).otherwise(None)
    )
    .withColumn(
        "is_matching",
        when(abs(col("diff_amount")) > 0.005, False).otherwise(True)
    )
    .select(
        "invoice_no",
        "subtotal",
        "tax",
        "total",
        "computed_total",
        "is_matching",
        "diff_amount",
        "diff_pct",
        "path"
    )
    .orderBy("invoice_no")
)

display(validated_df)

invoice_no,subtotal,tax,total,computed_total,is_matching,diff_amount,diff_pct,path
INV-1001,19000.00,2280.00,21280.00,21280.00,true,0.00,0.00,dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_1.pdf
INV-1003,16400.00,1968.00,18368.00,18368.00,true,0.00,0.00,dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_3.pdf
INV-1004,10700.00,909.50,11609.50,11609.50,true,0.00,0.00,dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_4.pdf
INV-1005,10500.00,787.50,11287.50,11287.50,true,0.00,0.00,dbfs:/Volumes/workspace/default/raw_invoices_pdf/sample_invoice_usd_2.pdf
